In [4]:
from MTNCI import MTNCI, Prediction
import sys
import torch
import numpy as np
from DatasetManager import DatasetManager, Filter
sys.path.append('./preprocessing/')
from utils import load_data_with_pickle, save_data_with_pickle
from CorpusManager import CorpusManager
from geoopt.optim import RiemannianAdam
import time


In [5]:
SOURCE_FILES_PATH = '/datahdd/vmanuel/MTNCI_datasets/source_files/'
# SOURCE_FILES_PATH = '../source_files/'

EMBEDDING_PATH = SOURCE_FILES_PATH + 'embeddings/'

FILE_ID = '16_3'

# PATH_TO_HYPERBOLIC_EMBEDDING = EMBEDDING_PATH + FILE_ID + 'final_tree_HyperE_MTNCI'
PATH_TO_HYPERBOLIC_EMBEDDING = EMBEDDING_PATH + FILE_ID + 'final_tree_HyperE_MTNCI_10'

PATH_TO_DISTRIBUTIONAL_EMBEDDING = EMBEDDING_PATH + FILE_ID + 'final_tree_type2vec_MTNCI'

CONCEPT_EMBEDDING_PATHS = [PATH_TO_DISTRIBUTIONAL_EMBEDDING, 
                           PATH_TO_HYPERBOLIC_EMBEDDING]

DATASET_PATH = SOURCE_FILES_PATH + 'vectors/'

X_PATH = DATASET_PATH + FILE_ID + 'X'
Y_PATH = DATASET_PATH + FILE_ID + 'Y'
ENTITIES_PATH = DATASET_PATH + FILE_ID + 'entities'

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type(torch.DoubleTensor)

In [8]:
datasetManager = DatasetManager(FILE_ID)
datasetManager.set_device(device)
datasetManager.load_entities_data(X_PATH = X_PATH,
                                  Y_PATH = Y_PATH,
                                  ENTITIES_PATH = ENTITIES_PATH)

-----------------------------
... loading entities data ...
entities data loaded in 8.54 seconds


In [9]:
entity_concepts_dict = load_data_with_pickle('./entity_concepts_dict_(multilabel).pkl')

In [10]:
entity_concepts_dict

defaultdict(list,
            {'airport': ['InformationAppliance',
              'TelevisionEpisode',
              'Device',
              'Film',
              'Airport'],
             'ibook': ['InformationAppliance', 'Device'],
             'aim': ['InformationAppliance', 'Weapon', 'Device'],
             'surface': ['InformationAppliance', 'Device'],
             'pluribus': ['InformationAppliance'],
             'timex computer': ['InformationAppliance', 'Device'],
             'univac': ['InformationAppliance'],
             'nintendo ds': ['InformationAppliance', 'VideoGame', 'Device'],
             'galerians rion': ['InformationAppliance', 'Cartoon', 'Anime'],
             'ectaco jetbook': ['InformationAppliance', 'Device'],
             'game boy color': ['InformationAppliance', 'VideoGame', 'Device'],
             'dornier do': ['InformationAppliance',
              'Aircraft',
              'Amphibian',
              'Ship',
              'MeanOfTransportation'],
        

In [29]:
i = 0
j = 0
for k, v in entity_concepts_dict.items():
    if len(v) == 1:
        i+=1
    j += 1
print(i)
print(j)

52536
94064


In [14]:
from tqdm.notebook import tqdm

new_X, new_Y, new_entities = [], [], []

bar = tqdm(total = len(datasetManager.X))

for x, y, e in zip(datasetManager.X, datasetManager.Y, datasetManager.entities):
    for concept in entity_concepts_dict[e]:
        new_X.append(x)
        new_Y.append(concept)
        new_entities.append(e + '_' + concept)
    bar.update(1)

In [17]:
save_data_with_pickle(X_PATH + '_multilabel', new_X)
save_data_with_pickle(Y_PATH + '_multilabel', new_Y)
save_data_with_pickle(ENTITIES_PATH + '_multilabel', new_entities)

In [18]:
len(new_X)

4601932